In [1]:
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from scipy import sparse

In [2]:
column = "word_seg"
train = pd.read_csv('../data/train_set.csv')
test = pd.read_csv('../data/test_set.csv')
test_id = test["id"].copy()

### tfidf提取特征

In [ ]:
vec_doc = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_doc = vec_doc.fit_transform(train[column])
test_term_doc = vec_doc.transform(test[column])

In [3]:
# sparse.save_npz('./trn_term_doc.npz', trn_term_doc)
# sparse.save_npz('./test_term_doc.npz', test_term_doc)

trn_term_doc = sparse.load_npz('./trn_term_doc.npz')
test_term_doc = sparse.load_npz('./test_term_doc.npz')

In [ ]:
vec_word = TfidfVectorizer(ngram_range=(1,1),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_word = vec_word.fit_transform(train[column])
test_term_word = vec_word.transform(test[column])

In [4]:
# sparse.save_npz('./trn_term_word.npz', trn_term_word)
# sparse.save_npz('./test_term_word.npz', test_term_word)

trn_term_word = sparse.load_npz('./trn_term_word.npz')
test_term_word = sparse.load_npz('./test_term_word.npz')

In [ ]:
vec_word_nramThree = TfidfVectorizer(ngram_range=(3,3),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_word_three = vec_word_nramThree.fit_transform(train[column])
test_term_word_three = vec_word_nramThree.transform(test[column])

In [5]:
# sparse.save_npz('./trn_term_word_three.npz', trn_term_word_three)
# sparse.save_npz('./test_term_word_three.npz', test_term_word_three)

trn_term_word_three = sparse.load_npz('./trn_term_word_three.npz')
test_term_word_three = sparse.load_npz('./test_term_word_three.npz')

In [ ]:
vec_word_nramfour = TfidfVectorizer(ngram_range=(4,4),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_word_four = vec_word_nramfour.fit_transform(train[column])
test_term_word_four = vec_word_nramfour.transform(test[column])

In [6]:
# sparse.save_npz('./trn_term_word_four.npz', trn_term_word_four)
# sparse.save_npz('./test_term_word_four.npz', test_term_word_four)

trn_term_word_four = sparse.load_npz('./trn_term_word_four.npz')
test_term_word_four = sparse.load_npz('./test_term_word_four.npz')

In [ ]:
vec_word_nramfive = TfidfVectorizer(ngram_range=(5,5),min_df=3, max_df=1.0,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='char')
trn_term_word_five = vec_word_nramfive.fit_transform(train[column])
test_term_word_five = vec_word_nramfive.transform(test[column])

In [ ]:
# sparse.save_npz('./trn_term_word_five.npz', trn_term_word_five)
# sparse.save_npz('./test_term_word_five.npz', test_term_word_five)
trn_term_word_five = sparse.load_npz('./trn_term_word_five.npz')
test_term_word_five = sparse.load_npz('./test_term_word_five.npz')

In [ ]:
vec_char = TfidfVectorizer(ngram_range=(1,2),min_df=3,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=10000,analyzer='char')
trn_char = vec_char.fit_transform(train["article"])
test_char = vec_char.transform(test["article"])

In [7]:
# sparse.save_npz('./trn_char.npz', trn_char)
# sparse.save_npz('./test_char.npz', test_char)

trn_char = sparse.load_npz('./trn_char.npz')
test_char = sparse.load_npz('./test_char.npz')

In [8]:
docLen  =pd.DataFrame( np.array(train[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
doclen_word = pd.DataFrame( np.array(train['article'].map(lambda x : len(x.split(" ")))/train[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))
docLen_test  =pd.DataFrame( np.array(test[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
doclen_word_test = pd.DataFrame( np.array(test['article'].map(lambda x : len(x.split(" ")))/test[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))

In [9]:
train_feaure =  hstack([trn_term_doc,trn_term_word,trn_term_word_three,trn_term_word_four,trn_char,doclen_word]).tocsr()
test_festure =  hstack([test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test]).tocsr()

In [ ]:
test_festure.shape
#

### 使用计数后tfidf,效果提升不大，维度上升60w,试过chi2降维，效果降低。，10折比5折有用

In [ ]:
# def feature_select(corpus, labels, k=1000):
#     """
#     select top k features through chi-square test
#     """
#     bin_cv = CountVectorizer(binary=True)
#     le = LabelEncoder()
#     X = bin_cv.fit_transform(corpus)
#     print("countVector Shap: ",X.shape)
#     y = le.fit_transform(labels).reshape(-1, 1)

#     k = min(X.shape[1], k)
#     skb = SelectKBest(chi2, k=k)
#     skb.fit(X, y)

#     feature_ids = skb.get_support(indices=True)
#     feature_names = bin_cv.get_feature_names()
#     vocab = {}

#     for new_fid, old_fid in enumerate(feature_ids):
#         feature_name = feature_names[old_fid]
#         vocab[feature_name] = new_fid

#     # we only care about the final extracted feature vocabulary
#     return vocab

### 降维

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler(with_mean=False)
ss.fit(train_feaure)
X_train_std = ss.transform(train_feaure)


In [ ]:
X_test_std = ss.transform(test_festure)

In [ ]:
skb = SelectKBest(chi2, k=840000)
train_feaure_chi = skb.fit_transform(X_train_std, y_all)

In [ ]:
test_feaure_chi = skb.transform(X_test_std)

In [ ]:
del X_test_std

In [10]:
y_all = (pd.Series(train['class'])-1).astype(int)

In [ ]:
# N = 2
# skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=42)
# xx_cv = []
# xx_pred = []
# for train_in,test_in in skf.split(train_feaure,y_all):
#     X_train,X_test,y_train,y_test =train_feaure[train_in],train_feaure[test_in],y_all[train_in],y_all[test_in]
#     lin_svm = svm.LinearSVC(C=1.0,random_state=2018,max_iter=5000)
#     lin_svm.fit(X_train,y_train)
#     print('Start predicting...')
#     preds = lin_svm.predict(X_test)
# #     y_pred = pd.Series(y_pred).map(lambda x:x+1)
#     fscoer = f1_score(y_test, preds, average='micro')
#     print(fscoer)
#     xx_cv.append(fscoer)
# #     xx_pred.append(lin_svm.predict(test_term_doc))
#     print(classification_report(y_test,preds))
#     print('-------------------end---------------------------')
# print('END:  ',np.mean(xx_cv))

In [11]:
X_train,X_test, y_train, y_test =train_test_split(train_feaure,y_all,test_size=0.3, random_state=0)

In [ ]:
# for i in [0.1,0.4,0.6,0.7,0.8,0.9]:
#     lin_svm = svm.LinearSVC(C=i,random_state=2018,max_iter=500)
#     lin_svm.fit(X_train,y_train)
#     print('Start predicting...     ',i)
#     preds = lin_svm.predict(X_test)
#     fscoer = f1_score(y_test, preds, average='micro')
#     print(fscoer)
#     print(classification_report(y_test,preds))

In [12]:
lr = LogisticRegression(C=6,dual=True)
lr.fit(X_train,y_train)
ypred = lr.predict_proba(X_test)
preds=np.argmax(ypred,axis=1)
fscoer = f1_score(y_test, preds, average='micro')
print(fscoer)
print(classification_report(y_test,preds))
#0.787

0.7872180941207144
             precision    recall  f1-score   support

          0       0.65      0.56      0.60      1639
          1       0.79      0.75      0.77       852
          2       0.88      0.88      0.88      2439
          3       0.86      0.83      0.85      1143
          4       0.84      0.76      0.80       697
          5       0.94      0.90      0.92      2114
          6       0.72      0.64      0.68       931
          7       0.69      0.77      0.73      2118
          8       0.93      0.94      0.93      2347
          9       0.71      0.70      0.70      1484
         10       0.67      0.73      0.70      1076
         11       0.69      0.67      0.68      1604
         12       0.74      0.79      0.77      2406
         13       0.77      0.81      0.79      1986
         14       0.91      0.90      0.90      2215
         15       0.80      0.67      0.73       936
         16       0.76      0.73      0.74       938
         17       0.86    

In [ ]:
ypred = lr.predict_proba(test_festure)
preds=np.argmax(ypred,axis=1)
# fscoer = f1_score(y_test, preds, average='micro')
# print(fscoer)
# print(classification_report(y_test,preds))

In [ ]:
res = pd.DataFrame()
res['id'] = test_id
res['class'] = pd.Series(preds).map(lambda x : x+1)
print(res.head())
res.to_csv('../data/baseline_lr.csv',header=True,index=False)

In [ ]:

# from sklearn.model_selection import StratifiedKFold
# N = 2
# from sklearn.preprocessing import OneHotEncoder
# skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=42)
# from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
# xx_cv = []
# xx_pred = []
# for train_in,test_in in skf.split(trn_term_doc,y_all):
#     X_train,X_test,y_train,y_test =trn_term_doc[train_in],trn_term_doc[test_in],y_all[train_in],y_all[test_in]
#     xgbc = XGBClassifier(
#         learning_rate =0.1,
#         n_estimators=500,
#         max_depth=6,
#         min_child_weight=0,
#         gamma=0,
#         subsample=0.7,
#         colsample_bytree=0.7,
#         objective= 'multi:softmax',
#         num_class=19,
#         eval_metric='merror',
#         nthread=-1,
#         scale_pos_weight=1,
#         seed=27)
#     print('Start predicting...')
#     xgbc_enc = OneHotEncoder()  
#     lin_svm = svm.LinearSVC(C=1.0,random_state=2018,max_iter=5000)
#     xgbc.fit(X_train,y_train)
#     xgbc_enc.fit(xgbc.apply(X_train)[:, :])
#     lin_svm.fit(xgbc_enc.transform(xgbc.apply(X_train)[:, :]), y_train)
#     y_xgbc_lr_test = lin_svm.predict(xgbc_enc.transform(xgbc.apply(X_test)[:,:]))
# #     preds=np.argmax(y_xgbc_lr_test,axis=1)
#     print(y_xgbc_lr_test)
# #     y_pred = pd.Series(y_pred).map(lambda x:x+1)
#     fscoer = f1_score(y_test, y_xgbc_lr_test, average='micro')
#     print(fscoer)
#     xx_cv.append(fscoer)
# #     xx_pred.append(lin_svm.predict(test_term_doc))
#     print(classification_report(y_test,y_xgbc_lr_test))
#     print('-------------------end---------------------------')
# print('END:  ',np.mean(xx_cv))

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# N = 3
# skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=42)
# from sklearn.metrics import roc_auc_score

# xx_cv = []
# xx_pred = []
# for train_in,test_in in skf.split(xtrain_scl,y_all):
#     X_train,X_test,y_train,y_test = xtrain_scl[train_in],xtrain_scl[test_in],y_all[train_in],y_all[test_in]
#     clf = MultinomialNB(alpha = 0.01)
#     clf.fit(X_train, y_train);
#     print('Start predicting...')
#     preds = clf.predict(X_test)
# #     y_pred = pd.Series(y_pred).map(lambda x:x+1)
#     fscoer = f1_score(y_test, preds, average='micro')
#     print(fscoer)
#     xx_cv.append(fscoer)
# #     xx_pred.append(lin_svm.predict(test_term_doc))
#     print(classification_report(y_test,preds))
#     print('-------------------end---------------------------')
# print('END:  ',np.mean(xx_cv))